<a href="https://colab.research.google.com/github/ommestriker007/Time_series/blob/main/time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geemap.geemap as geemap
import ee
import os


In [ ]:
ee.Authenticate()
ee.Initialize(project = 'neon-feat-313910')

base = geemap.Map()
# base

In [ ]:
India = ee.FeatureCollection('projects/neon-feat-313910/assets/National_Boundary')
base.addLayer(India, {}, 'India')
# base

In [ ]:
region = ee.Geometry.Polygon(
    [[[67.060547, 4.740675],
     [67.060547, 38.065392],
     [98.964844, 38.065392],
     [98.964844, 4.740675],
     [67.060547, 4.740675]]],
    None,
    False)

In [ ]:
NDVI = ee.ImageCollection("MODIS/061/MOD13A2").select('NDVI')
base.addLayer(NDVI, {}, 'veg_indices')
# base

In [ ]:
NDVI_India = NDVI.map(lambda img: img.set('doy', ee.Date(img.get('system:time_start')).getRelative('day', 'year')));

In [ ]:
distinctDOY = NDVI_India.filterDate('2010-01-01', '2011-01-01');

In [ ]:
filter = ee.Filter.equals(leftField = 'doy', rightField = 'doy')
join = ee.Join.saveAll('doy_matches')
joincol = ee.ImageCollection(join.apply(distinctDOY, NDVI_India, filter))

In [ ]:
comp = joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()).set('doy', ee.Number(img.get('doy'))))

In [ ]:
vis_params = {
  'min': 0,
  'max': 9000,
  'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ],
};

In [ ]:
rgbvis = comp.map(lambda img: img.visualize(bands = ['NDVI_mean'], **vis_params).clip(India))

In [ ]:
gifParams = {
  'region': region,
  'dimensions': 600,
  'crs': 'EPSG:4326',
  'framesPerSecond': 10,
  'format':'gif'
};

print(rgbvis.getVideoThumbURL(gifParams));

https://earthengine.googleapis.com/v1/projects/neon-feat-313910/videoThumbnails/005a0669493fa678c786b1dc87077688-be45a7eee2ddf5c75ee5dddc73f47fb6:getPixels
